In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error as MSE,mean_absolute_error as MAE,r2_score as r2
from sklearn.model_selection import train_test_split

In [ ]:
data=pd.read_csv("/content/mission 4.csv")

In [ ]:
data.head(5)

In [ ]:
data.info()

In [ ]:
data.isnull().sum()

In [ ]:
data.describe()

In [ ]:
data.columns

In [ ]:
le={}
col=['Brand', 'Processor Type','Operating System']
for i in col:
  le[i]=LabelEncoder()
  data[i]=le[i].fit_transform(data[i])


In [ ]:
data.head()

In [ ]:
plt.figure(figsize=(10,6))
sns.histplot(x=data["Price ($)"],color="black",kde=True)
plt.title("relation between target and feature")
plt.xlabel("Price ($)")
plt.ylabel("Frequency")
plt.show()

In [ ]:
plt.figure(figsize=(10,6))
sns.scatterplot(x=data["Price ($)"],y=data["Brand"],color="black")
plt.title("relation between target and feature")
plt.xlabel("Price ($)")
plt.ylabel("Brand")
plt.show()

In [ ]:
plt.figure(figsize=(10,6))
sns.barplot(x=data["Price ($)"],y=data["Processor Type"],color="black")
plt.title("relation between target and feature")
plt.xlabel("Price ($)")
plt.ylabel("Processor Type")
plt.show()

In [ ]:
min=MinMaxScaler()
data[['RAM Size (GB)', 'Storage (GB)','Screen Size (inches)']]=min.fit_transform(data[['RAM Size (GB)', 'Storage (GB)','Screen Size (inches)']])

In [ ]:
x=data.drop("Price ($)",axis=1)
y=data["Price ($)"]
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

In [ ]:
lr=LinearRegression()
lr.fit(x_train,y_train)
y_pred_lr=lr.predict(x_test)
print(MSE(y_test,y_pred_lr))
print(MAE(y_test,y_pred_lr))
print(r2(y_test,y_pred_lr))

In [ ]:
dt=DecisionTreeRegressor()
dt.fit(x_train,y_train)
y_pred_dt=dt.predict(x_test)
print(MSE(y_test,y_pred_dt))
print(MAE(y_test,y_pred_dt))

In [ ]:
rf=RandomForestRegressor()
rf.fit(x_train,y_train)
y_pred_rf=rf.predict(x_test)
print(MSE(y_test,y_pred_rf))
print(MAE(y_test,y_pred_rf))

In [ ]:
params={'max_depth':[10,20,30]}
grid=GridSearchCV(DecisionTreeRegressor(),params,cv=5)
grid.fit(x_train,y_train)
print(grid.best_params_)
grid_pred=grid.best_estimator_.predict(x_test)
print(MSE(y_test,grid_pred))

In [ ]:
param={'max_leaf_nodes':[10,20,30]}
random=RandomizedSearchCV(RandomForestRegressor(),param,cv=5)
random.fit(x_train,y_train)
print(random.best_params_)
random_pred=random.best_estimator_.predict(x_test)
print(MSE(y_test,random_pred))


In [ ]:
plt.figure(figsize=(10,6))
sns.scatterplot(x=y_test,y=grid_pred,color="black")
sns.scatterplot(x=y_test,y=random_pred,color="green")
plt.title("relation between target and feature")
plt.xlabel("y_test")
plt.ylabel("y_pred")
plt.show()

In [ ]:
import gradio as gr

In [ ]:
data.columns

In [ ]:
def haneen(brand,processor,ram_size,storage,screen_size,operation_system):
  try:
    input_data=pd.DataFrame({
        "Brand":[brand],
        "Processor Type":[processor],
        "RAM Size (GB)":[ram_size],
        "Storage (GB)":[storage],
        "Screen Size (inches)":[screen_size],
        "Operating System":[operation_system]
    })
    for i in col:
      input_data[i]=le[i].transform(input_data[i])
    input_data[['RAM Size (GB)', 'Storage (GB)','Screen Size (inches)']]=min.transform(input_data[['RAM Size (GB)', 'Storage (GB)','Screen Size (inches)']])
    result=rf.predict(input_data)
    print(result)
    return result[0]
  except Exception as e:
    return str(e)
gr.Interface(
    fn=haneen,
    inputs = [
        gr.Dropdown(["Acer","Apple","HP","Lenovo"],label="Brand"),
        gr.Dropdown(["Intel Core i5","Intel Core i7","AMD Ryzen 5","AMD Ryzen 7"],label="Processor Type"),
        gr.Number(label="RAM Size (GB)"),
        gr.Number(label="Storage (GB)"),
        gr.Number(label="Screen Size (inches)"),
        gr.Dropdown(["macOS","Linux","Windows"],label="Operating System")
    ],
     outputs=gr.Number(label="prediction")
).launch()
